In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1. Chargement des données pré-nettoyées
# (Fichier contenant uniquement les réponses valides, avec l'échelle ajustée de 1 à 5)
file_path = "données_nettoyées_survey.csv"
df = pd.read_csv(file_path)

# 2. Création des scores composites pour les variables indépendantes
# On calcule la moyenne des réponses de chaque participant pour chaque dimension
cols_trust = ['Q7_1', 'Q7_2', 'Q7_3', 'Q7_4']
cols_privacy = ['Q8_1', 'Q8_2', 'Q8_3']
cols_ethics = ['Q9_1', 'Q9_2', 'Q9_3']
cols_env = ['Q10_1', 'Q10_2', 'Q10_3']

df['Confiance'] = df[cols_trust].mean(axis=1)
df['Vie_Privee'] = df[cols_privacy].mean(axis=1)
df['Ethique'] = df[cols_ethics].mean(axis=1)
df['Environnement'] = df[cols_env].mean(axis=1)

# 3. Création de la variable dépendante : Intensité d'adoption (Y)
# Fonction pour compter le nombre de contextes d'usage cochés à la question Q5 (ex: "Travail, Personnel")
def get_diversity_score(x):
    if pd.isna(x):
        return 0
    return len(str(x).split(','))

df['Q5_Score'] = df['Q5'].apply(get_diversity_score)

# L'intensité est la somme de la fréquence (Q4) et de la diversité des contextes (Q5)
df['Adoption_Intensite'] = df['Q4'] + df['Q5_Score']

# 4. Préparation de la table finale pour la régression
# On isole les 5 variables du modèle et on exclut automatiquement les lignes avec des données manquantes
analysis_df = df[['Adoption_Intensite', 'Confiance', 'Vie_Privee', 'Ethique', 'Environnement']].dropna()

# 5. Calcul et affichage de la Matrice de Corrélation
corr_matrix = analysis_df.corr()
print("--- MATRICE DE CORRÉLATION ---")
print(corr_matrix.round(3))
print("\n")

# 6. Exécution du modèle de Régression Linéaire Multiple (OLS)
# X (Variables indépendantes) et Y (Variable dépendante)
X = analysis_df[['Confiance', 'Vie_Privee', 'Ethique', 'Environnement']]
X = sm.add_constant(X) # Ajout de l'ordonnée à l'origine (Intercept)
y = analysis_df['Adoption_Intensite']

# Ajustement du modèle et affichage des résultats
model = sm.OLS(y, X).fit()
print("--- RÉSULTATS DE LA RÉGRESSION OLS ---")
print(model.summary())

--- MATRICE DE CORRÉLATION ---
                    Adoption_Intensite  Confiance  Vie_Privee  Ethique  \
Adoption_Intensite               1.000      0.241      -0.072    0.107   
Confiance                        0.241      1.000       0.075    0.227   
Vie_Privee                      -0.072      0.075       1.000    0.412   
Ethique                          0.107      0.227       0.412    1.000   
Environnement                    0.205     -0.067       0.218    0.182   

                    Environnement  
Adoption_Intensite          0.205  
Confiance                  -0.067  
Vie_Privee                  0.218  
Ethique                     0.182  
Environnement               1.000  


--- RÉSULTATS DE LA RÉGRESSION OLS ---
                            OLS Regression Results                            
Dep. Variable:     Adoption_Intensite   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least 